# Website Summarizer


In [24]:
# imports

import os
import requests
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import time
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [25]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [26]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [27]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url

        userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        options = Options()
        options.add_argument(f"user-agent={userAgent}")
        options.add_argument("--headless")
        
        driver = webdriver.Chrome(options = options)#, executable_path= "C:/Users/r_sab/chromedriver.exe")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [28]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes cryptos and can identify anamolous tokens in a website \
and provides a short summary of the potential opportunities \
Respond in markdown."

In [29]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of potential opportunities \n\n"
    user_prompt += website.text
    return user_prompt

In [30]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [31]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [32]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [33]:
display_summary("https://www.coingecko.com/")

# Summary of Potential Opportunities in Cryptocurrency

## Market Overview
- **Total Cryptocurrencies**: 16,990
- **Market Cap**: $3.044 Trillion (up 1.4%)
- **24h Trading Volume**: $151.081 Billion
- **Bitcoin (BTC) Dominance**: 61.3%
- **Ethereum (ETH) Dominance**: 7.16%

## Notable Gainers
- **Zerebro (ZEREBRO)**: 196.58% increase
- **Neiro on ETH**: 78.1%
- **Finvesta**: 58.3%
- **Official Trump (TRUMP)**: 45.5% increase

## Trending Tokens
- **Zora (ZORA)**: 33.2% increase
- **Sui (SUI)**: 23.63% increase

## Anomalous Tokens and Insights
1. **Zerebro (ZEREBRO)** - A significant gain of **196.58%** indicates potential speculative interest or market corrections. An ideal token for traders looking for high-risk, high-reward opportunities.
   
2. **Official Trump (TRUMP)** - Surged by **45.5%**, showcasing community interest. It may benefit from political sentiment but is likely volatile due to its branding.

3. **Sui (SUI)** - Despite being larger, a **23.63%** rise can suggest ongoing development transitions or partnerships that could be worth exploring for medium-term investments.

4. **Diverse Ecosystem Tokens** - Notable mentions in the Polkadot and other ecosystem categories might indicate strong development activity and future growth potential, making them worth tracking.

## Investment Strategies
- **Focus on Gainers**: Investors might consider allocating funds to tokens that have shown substantial short-term gains, but should assess the sustainability of growth.
- **Monitor Market Trends**: Tools available on CoinGecko such as comparison charts and API access to market data provide insights needed to navigate this volatile market.
- **Diverse Portfolio**: Given the varied performance among different categories, diversifying investments across large-cap, mid-cap, and emerging tokens can balance risk and enhance potential returns.

## Conclusion
The cryptocurrency market shows a dynamic landscape with considerable growth opportunities in certain tokens and valuable data at your disposal. Staying updated and utilizing tools can help in making informed investment decisions.